# Model Building

sources:

- https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
- https://towardsdatascience.com/generating-haiku-with-deep-learning-dbf5d18b4246
- https://www.analyticsvidhya.com/blog/2020/01/first-text-classification-in-pytorch/

In [144]:
import os
import sys
import pickle
import torch
import torch.nn
import torch.optim
import torch.utils.data

In [145]:
"""Google Drive"""
from google.colab import drive
drive.mount('/content/gdrive/')
root_path = '/content/gdrive/My Drive/Virginia Tech/graduate/courses/2021_spring/ece_5424/assignments/project_ece_5424/'
dataset_path = './dataset'
store_file = os.path.join(root_path,dataset_path,'lyrics.pickle')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [146]:
"""Offline Usage"""
# dataset_path = '../../dataset'
# store_file = os.path.join(dataset_path,'lyrics.pickle')

'Offline Usage'

## Construct Dataset class

In [147]:
class WorshipLyricDataset(torch.utils.data.Dataset):
    """Worhip Song dataset from Genius.
    """

    def __init__(self, path: str):

        # Load the pre-processed pickle file.
        with open(path, 'rb') as fp:
            store = pickle.load(fp)
        
        # Unpack the pickle.
        self.index2token = store['index2token']
        self.token2index = store['token2index']
        self.counts = store['counts']
        self.corpus = store['corpus']
        
        # n_words = len(self.index2token)
        # self.vectors = [torch.nn.functional.one_hot(torch.LongTensor(vec), num_classes=n_words) for vec in store['vectors']]
        self.vectors = [torch.LongTensor(vec) for vec in store['vectors']]
        self.syllables = torch.nn.functional.one_hot(torch.LongTensor(store['syllables'])) # One-hot encoded syllabl counts.

    def __len__(self):
        return len(self.vectors)

    def __getitem__(self, idx):
        # lyric = {
        #     'vector': self.vectors[idx],
        #     'syllables': self.syllables[idx],
        # }
        return (self.vectors[idx], self.syllables[idx],)

In [148]:
# Construct the data object.
dataset = WorshipLyricDataset(path=store_file)

In [149]:
def pad_collate(batch):
    """Pad batches from dataloader.

    This allows for more efficient padding,
    by only padding within each batch.
    """
    sentences, syllables = zip(*batch)
    sen_lens = torch.LongTensor([len(vec) for vec in sentences])
    sen_pad = torch.nn.utils.rnn.pad_sequence(sentences, batch_first=True, padding_value=0)
    syllables = torch.stack(syllables) # Convert tuple of tensors to single 2D tensor.
    syllables = syllables.reshape(syllables.size(0),1,syllables.size(1)) # Convert to 3D.
    syllables = syllables.repeat_interleave(sen_pad.size(1), dim=1) # Duplicate syllable count for every word in each sentence.
    # print('pad_collate','sen_lens',sen_lens.size())
    # print('pad_collate','sen_pad',sen_pad.size())
    # print('pad_collate','syllables',syllables.size())
    return (sen_pad,syllables,sen_lens,)

In [150]:
# Construct data loader.
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2, collate_fn=pad_collate)

## Construct Model

We use an encode/decode architecture, with **encoder** and **decoder** layers, and also add an **attention** layer.

This architecture was adapted from the wondeful PyTorch tutorial ["NLP From Scratch: Translation with a Sequence to Sequence Network and Attention"](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html).

sources:
- https://medium.com/@stepanulyanin/captioning-images-with-pytorch-bc592e5fd1a3
- https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
- https://suzyahyah.github.io/pytorch/2019/07/01/DataLoader-Pad-Pack-Sequence.html

### Encoder
The encoder outputs a value for each word in a given input sentence. For each input word, the encoder outputs the value vector and a hidden state, the hidden state is used for the next input word.

In [151]:
class SentenceRNN(torch.nn.Module):
    def __init__(self, n_hidden: int, n_vocab: int, n_layers: int, dropout: float = 0., bidirectional: bool = False):
        super().__init__()

        self.n_hidden = n_hidden
        self.n_vocab = n_vocab
        self.n_layers = n_layers
        self.bidirectional = bidirectional
        self.n_dir = 2 if bidirectional else 1

        # Embedding layer.
        self.embed = torch.nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=n_hidden,
        )

        # LSTM layer.
        self.lstm = torch.nn.LSTM(
            input_size=n_hidden,
            hidden_size=n_hidden,
            num_layers=n_layers,
            dropout=dropout,
            bidirectional=bidirectional,
            batch_first=True,
            )

        # Word mapping fully-connected layer.
        self.fc = torch.nn.Linear(in_features=n_hidden, out_features=n_vocab)
    
    def forward(self, sentences: torch.Tensor, lens: torch.Tensor, hidden: torch.Tensor, cell: torch.Tensor):
        """
        Args:
            features (torch.Tensor): Embedded syllable features.
            sentences (torch.Tensor): Sentence word vectors.
            lens (torch.Tensor): True lengths of padded sentence vectors.
            hidden (torch.Tensor): Hidden state vector.
            cell (torch.Tensor): Cell state vector.
        """
        
        # Embed the sentence vectors as floating-point.
        #
        # inputs: (batch_size, sentence_length,)
        sentences_embed = self.embed(sentences)
        # embedded: (batch_size, sentence_length, embed_dim,)
        # print('sentences_embed',sentences_embed.size())

        # # Reshape the embedding so that it will fit into the LSTM.
        # sentences_embed = sentences_embed.view(sentences_embed.size(0),sentences_embed.size(1), -1)
        # print('sentences_embed',sentences_embed.size())

        # Pack the embedding so that the paddings are ignored.
        sentences_embed_packed = torch.nn.utils.rnn.pack_padded_sequence(
            input=sentences_embed,
            lengths=lens, 
            batch_first=True,
            enforce_sorted=False,
            )
        # print('sentences_embed_packed','data',sentences_embed_packed.data.size())

        # Pass the input feature vector as the first step.
        output_packed, (hidden, cell) = self.lstm(sentences_embed_packed, (hidden,cell,))
        # print('output_packed',output_packed.data.size())

        # Get padded output
        output_padded, output_lens = torch.nn.utils.rnn.pad_packed_sequence(output_packed, batch_first=True)
        # print('output_padded',output_padded.size())

        # Obtain word-level classification.
        output_padded_fc = self.fc(output_padded)
        # print('output_padded_fc',output_padded_fc.size())

        # Run packing on output layer.
        # return output_padded, output_lens, hidden
        return output_padded_fc, output_lens, (hidden, cell,)

    def init_hc(self, batch_size: int, device: str = 'cpu'):
        return torch.zeros((self.n_layers*self.n_dir, batch_size, self.n_hidden), device=device)

## Train

In [152]:
import time
from contextlib import contextmanager

class timecontext:
    """Elapsed time context manager."""
    def __enter__(self):
        self.seconds = time.time()
        return self
    
    def __exit__(self, type, value, traceback):
        self.seconds = time.time() - self.seconds

@contextmanager
def timecontextprint(description='Elapsed time'):
    """Context manager to print elapsed time from call."""
    with timecontext() as t:
        yield t
    print(f"{description}: {t.seconds} seconds")

In [153]:
import random
def train(decoder, loader, epochs, optimizer_decoder, criterion, device='cpu', teacher_force_ratio=0.5):
    decoder.to(device)

    decoder.train()

    for e in range(epochs):
        running_loss = 0.0
        for b,batch in enumerate(loader):
            sentences,syllables,sen_lens = batch
            sentences = sentences.to(device)
            syllables = syllables.to(device)
            sen_lens = sen_lens.to(device)

            # Initialize hidden output.
            decoder_hidden = decoder.init_hc(32, device=device)
            decoder_cell = decoder.init_hc(32, device=device)

            # Decode.
            # outputs, out_lens, _ = decoder(features, sentences, sen_lens, decoder_hidden, decoder_cell)
            SOS_token = dataset.token2index['<sos>']
            # decoder_input = SOS_token*torch.ones((sentences.size(0), sentences.size(1), 1,), dtype=torch.long, device=device)
            decoder_input = SOS_token*torch.ones((sentences.size(0), 1,), dtype=torch.long, device=device)
            # decoder_input_lens = torch.ones((sentences.size(0),), dtype=torch.long, device=device)
            decoder_input_lens = torch.ones((sentences.size(0),), dtype=torch.long)

            # print('decoder_input',decoder_input.size())
            # print('decoder_input_lens',decoder_input_lens.size())

            
            loss = 0

            use_teacher_forcing = True if random.random() < teacher_force_ratio else False

            # Teacher forcing.
            # Feed the target as the next input.
            if use_teacher_forcing:
                for target_idx in range(1, sentences.size(1)):
                    outputs, out_lens, (decoder_hidden, decoder_cell) = decoder(decoder_input, decoder_input_lens, decoder_hidden, decoder_cell)

                    # Reshape outputs and targets to fit insize the criterion.
                    outputs = outputs.squeeze(dim=1)
                    targets = sentences[:,target_idx].reshape(sentences.size(0),-1)

                    # print(f"[target {target_idx}] {targets.squeeze(dim=1)}")

                    # Calculate batch loss.
                    loss += criterion(
                        outputs,
                        targets.squeeze(dim=1),
                    )

                    # For teacher forcing set the input of the
                    # next round to be the current target.
                    decoder_input = targets.detach()

            # No teacher forcing.
            # Feed the RNN predictions as the next input.
            else:
                for target_idx in range(1, sentences.size(1)):
                    outputs, out_lens, (decoder_hidden, decoder_cell) = decoder(decoder_input, decoder_input_lens, decoder_hidden, decoder_cell)

                    # Get best prediction.
                    topv, topi = outputs.topk(1)

                    # Reshape outputs and targets to fit insize the criterion.
                    outputs = outputs.squeeze(dim=1)
                    targets = sentences[:,target_idx].reshape(sentences.size(0),-1)

                    # Calculate batch loss.
                    loss += criterion(
                        outputs,
                        targets.squeeze(dim=1),
                    )

                    # Set the input of the next round to be the current prediction.
                    decoder_input = topi.squeeze(dim=-1).detach()

            # Back-propagate, and step the optimizers.
            # if target_idx == 1: retain_graph = True
            # else: retain_graph = False
            loss.backward()
            optimizer_decoder.step()
            
            # Zero the gradients
            optimizer_decoder.zero_grad()

            # Accumulate the loss for this epoch.
            running_loss += loss.item()

            # print(f"[batch {b}] loss: {running_loss}")

            # topv, topi = outputs.topk(1)
            # for i in range(5):
            #     s = ' '.join([dataset.index2token[idx] for idx in topi[i]])
            #     print(topi[i].view(-1))
            #     print(s)
            #     print()

        # Report epoch results.
        print(f'Epoch {e}: loss {running_loss}')

In [154]:
# Length of vocabulary.
n_words = len(dataset.index2token)

# Decoder.
decoder = SentenceRNN(
    n_vocab=n_words,
    n_hidden=128,
    n_layers=1,
    dropout=0.1,
    bidirectional=False,
)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [155]:
# Set runtime device.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cuda


In [156]:
# Define path to decoder model storage.
decoder_store = os.path.join(root_path, dataset_path, 'decoder.pt')

In [157]:
load_from_file = True

# Load model from store file.
if load_from_file and os.path.exists(decoder_store):
    decoder.load_state_dict(torch.load(decoder_store))

else:
    # Learning parameters.
    epochs = 10
    lr = 1e-3
    print(f'Training decoder model: epochs={epochs}, lr={lr}, batches={len(dataloader)}')

    # Train the model.
    # Display training time too.
    with timecontextprint():
        optim_decoder = torch.optim.Adam(decoder.parameters(), lr=lr)
        criterion = torch.nn.CrossEntropyLoss(reduction='mean')
        train(decoder,
            loader=dataloader,
            epochs=epochs,
            optimizer_decoder=optim_decoder,
            criterion=criterion,
            device=device,
        )

    # Store model state to file.
    decoder_store = os.path.join(root_path,dataset_path,'decoder.pt')
    torch.save(decoder.state_dict(), decoder_store)
    print(f'Saved decoder model: {decoder_store}')

Training decoder model: epochs=10, lr=0.001, batches=898
Epoch 0: loss 36063.487173080444
Epoch 1: loss 32784.56394195557
Epoch 2: loss 31954.298318862915
Epoch 3: loss 31770.73614692688
Epoch 4: loss 31322.788995742798
Epoch 5: loss 31091.779554367065
Epoch 6: loss 30705.09156036377
Epoch 7: loss 30519.26244354248
Epoch 8: loss 30194.965978622437
Epoch 9: loss 30001.43767929077
Elapsed time: 222.98313546180725 seconds
Saved decoder model: /content/gdrive/My Drive/Virginia Tech/graduate/courses/2021_spring/ece_5424/assignments/project_ece_5424/./dataset/decoder.pt


## Evaluate

In [158]:
import numpy as np

def evaluate(decoder, seed='<sos>', max_length=None, device='cpu'):
    decoder.to(device)
    decoder.eval()

    with torch.no_grad():

        # Initialize hidden output.
        decoder_hidden = decoder.init_hc(1, device=device)
        decoder_cell = decoder.init_hc(1, device=device)

        # Decode.
        EOS_token = dataset.token2index['<eos>']
        SOS_token = dataset.token2index['<sos>']
        seed_token = dataset.token2index.get(seed, SOS_token)
        decoder_input = seed_token*torch.ones((1, 1,), dtype=torch.long, device=device)
        decoder_input_lens = torch.ones((1,), dtype=torch.long)


        decoded_tokens = [seed]
        i = 0
        while True:
            outputs, out_lens, (decoder_hidden, decoder_cell) = decoder(decoder_input, decoder_input_lens, decoder_hidden, decoder_cell)

            probs = np.array(torch.nn.functional.softmax(outputs, dim=2).squeeze().cpu())
            probs = probs / probs.sum()
            topi = torch.tensor(np.random.choice(decoder.n_vocab, 1, p=probs, replace=False), dtype=torch.long, device=device).view(1,1,1)

            if topi.item() == EOS_token:
                decoded_tokens.append('<eos>')
                break
            else:
                decoded_tokens.append(dataset.index2token[topi.item()])

            # Stop if maximum sentence length has been reached.
            if max_length and len(decoded_tokens) == max_length:
                break

            # Set the input of the next round to be the current prediction.
            decoder_input = topi.squeeze(dim=-1).detach()
            i += 1

        return decoded_tokens

In [167]:
def generate_song(num_lines: int, max_length: int = None, seeds: list = None) -> str:
    lines = []
    seed = '<sos>'
    if not seeds:
        seeds = [seed]*num_lines
    for i in range(num_lines):
        tokens = evaluate(decoder, max_length=max_length, seed=seeds[i], device=device)
        if tokens[0] != '<sos>': tokens = ['<sos>'] + tokens
        lines.append(' '.join(tokens))
    return '\n'.join(lines)

In [169]:
print(generate_song(num_lines=3, max_length=10))

<sos> have joy and rest with me tell this higher
<sos> am not running back to rise <eos>
<sos> lord inside me home <eos>


In [170]:
print(generate_song(num_lines=3, seeds=['god', 'we', 'they']))

<sos> god want that love is work yeah yeah <eos>
<sos> we write hoping caught warning yet as not give me go <eos>
<sos> they blown human searching defeated preaching blew is the clear <eos>


In [171]:
print(generate_song(num_lines=3, seeds=['we', 'they', 'you']))

<sos> we cry prone out of itself <eos>
<sos> they would really remember just blind <eos>
<sos> you planned sung let me go to you <eos>
